In [128]:
import os
import sys
from pathlib import Path
import cv2
import numpy as np


def setup_project_path():
    current = Path.cwd()
    while not (current / 'craft').exists():
        current = current.parent
    return current
project_root = setup_project_path()
sys.path.insert(0, str(project_root))

In [129]:
def imwrite_unicode(path, img):
    ext = os.path.splitext(path)[1]
    ok, buf = cv2.imencode(ext, img)
    if not ok:
        return False
    with open(path, "wb") as f:
        f.write(buf.tobytes())
    return True

In [130]:
def imread_unicode(path, flags=cv2.IMREAD_COLOR):
    try:
        with open(path, "rb") as f:
            data = f.read()
        img_array = np.frombuffer(data, np.uint8)
        img = cv2.imdecode(img_array, flags)
        return img
    except Exception as e:
        print("[imread_unicode ERROR]", e)
        return None


In [131]:
def load_centers_from_file(centers_file: str):
    centers = []
    with open(centers_file, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            x_str, y_str = line.split(",")
            centers.append((float(x_str), float(y_str)))
    return centers


In [132]:
def load_tilts_from_file(tilt_file: str):
    tilts = []
    with open(tilt_file, "r", encoding="utf-8") as f:
        for line in f:
            s = line.strip()
            if not s:
                continue
            tilts.append(float(s))
    return tilts

In [133]:
def load_spacings_from_file(spacing_file: str):
    items = []
    with open(spacing_file, "r", encoding="utf-8") as f:
        for line in f:
            s = line.strip()
            if not s:
                continue
            parts = s.split(",")
            if len(parts) != 4:
                raise ValueError(f"Invalid spacing line: {s}")
            i1 = int(parts[0])
            i2 = int(parts[1])
            spacing = float(parts[2])
            flag = parts[3].strip()
            items.append({"i1": i1, "i2": i2, "spacing": spacing, "flag": flag})
    return items


In [134]:
def draw_label(img, text, x, y, scale=0.3, thickness=1, pad=3):
    text = str(text)
    (tw, th), baseline = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, scale, thickness)
    x, y = int(x), int(y)

    x = max(0, min(x, img.shape[1]-1))
    y = max(th + pad, min(y, img.shape[0]-1))

    cv2.rectangle(img, (x, y - th - pad), (x + tw + pad*2, y + baseline + pad), (0, 0, 0), -1)
    cv2.putText(img, text, (x + pad, y), cv2.FONT_HERSHEY_SIMPLEX, scale, (255, 255, 255),
                thickness, cv2.LINE_AA)

In [135]:
def _put_text(
    img, text, x, y,
    scale=0.6, thickness=2,
    fg_color=(255,255,255), bg_color=(0,0,0),
    pad=3,
    anchor="lt",  
):
    text = str(text)
    (tw, th), baseline = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, scale, thickness)

    x, y = float(x), float(y)

    if anchor == "center":
        x = x - (tw + pad*2) / 2.0
        y = y + (th + pad) / 2.0 

    x, y = int(x), int(y)

    # 배경 박스
    cv2.rectangle(
        img,
        (x, y - th - pad),
        (x + tw + pad*2, y + baseline + pad),
        bg_color,
        -1,
    )

    # 글씨
    cv2.putText(
        img,
        text,
        (x + pad, y),
        cv2.FONT_HERSHEY_SIMPLEX,
        scale,
        fg_color,
        thickness,
        cv2.LINE_AA,
    )


In [136]:
def visualize(
    image_path: str,
    centers_file: str,
    out_path: str,
    tilt_file: str | None = None,
    spacing_file: str | None = None,
    *,
    show_center: bool = True,
    show_tilt: bool = True,
    show_space: bool = True,
    sort_by_x: bool = True,
    draw_pair_lines: bool = True,      
    center_radius: int = 4,
):

    img = imread_unicode(image_path)
    if img is None:
        raise FileNotFoundError(f"Cannot read image: {image_path}")

    centers = load_centers_from_file(centers_file)
    if sort_by_x:
        centers = sorted(centers, key=lambda p: p[0])

    # 1) center 표시
    if show_center:
        for (cx, cy) in centers:
            cv2.circle(img, (int(cx), int(cy)), center_radius, (0, 255, 0), -1)
            cv2.circle(img, (int(cx), int(cy)), center_radius + 2, (0, 0, 0), 2)
            draw_label(img, f"({cx:.1f},{cy:.1f})", cx + 6, cy - 6, scale=0.45, thickness=1)

    # 2) tilt 표시
    if show_tilt:
        if tilt_file is None or not os.path.exists(tilt_file):
            pass
        else:
            tilts = load_tilts_from_file(tilt_file)
            pair_count = max(0, len(centers) - 1)
            m = min(pair_count, len(tilts))

            for i in range(m):
                (x1, y1) = centers[i]
                (x2, y2) = centers[i + 1]

                if draw_pair_lines:
                    cv2.line(img, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)

                mx = (x1 + x2) / 2.0
                my = (y1 + y2) / 2.0

                _put_text(
                    img,
                    f"{tilts[i]:.2f}°",
                    mx,
                    my - 6,
                    scale=0.3,
                    thickness=1,
                    fg_color=(255, 255, 255), 
                    bg_color=(0, 0, 0),  
                    anchor="center",
                )



    # 3) space 표시 
    if show_space:
        if spacing_file is None or not os.path.exists(spacing_file):
            pass
        else:
            items = load_spacings_from_file(spacing_file)

            for it in items:
                i1 = it["i1"]
                i2 = it["i2"]
                spacing = it["spacing"]
                flag = it["flag"]

                if i1 < 0 or i2 >= len(centers) or i2 != i1 + 1:
                    continue

                (x1, y1) = centers[i1]
                (x2, y2) = centers[i2]

                if draw_pair_lines:
                    color = (0, 165, 255) if flag == "gap" else (255, 0, 255)
                    cv2.line(img, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)

                mx = (x1 + x2) / 2.0
                my = (y1 + y2) / 2.0

                label = f"{spacing:.1f}"
                if flag == "gap":
                    label += " gap"

                _put_text(
                    img,
                    label,       
                    mx,       
                    my + 8,     
                    scale=0.3,
                    thickness=1,
                    fg_color=(255, 255, 255),
                    bg_color=(0, 0, 0),
                    anchor="center",

                )



    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    imwrite_unicode(out_path, img)


In [137]:
def visualize_folder(
    image_folder: str,
    centers_folder: str,
    out_folder: str,
    *,
    tilts_folder: str | None = None,
    spacing_folder: str | None = None,
    show_center: bool = True,
    show_tilt: bool = True,
    show_space: bool = True,
):

    os.makedirs(out_folder, exist_ok=True)

    for fname in os.listdir(centers_folder):
        if not fname.startswith("res_") or not fname.endswith("_center.txt"):
            continue

        name = fname[len("res_") : -len("_center.txt")]
        centers_file = os.path.join(centers_folder, fname)

        img_path = None
        for ext in [".jpg", ".jpeg", ".png", ".bmp", ".webp"]:
            cand = os.path.join(image_folder, name + ext)
            if os.path.exists(cand):
                img_path = cand
                break

        if img_path is None:
            print(f"[WARN] image not found for '{name}' in {image_folder}")
            continue

        tilt_file = None
        if tilts_folder is not None:
            tilt_file = os.path.join(tilts_folder, f"res_{name}_angle.txt")

        spacing_file = None
        if spacing_folder is not None:
            spacing_file = os.path.join(spacing_folder, f"res_{name}_spacing.txt")

        out_path = os.path.join(out_folder, f"vis_{name}.jpg")

        try:
            visualize(
                image_path=img_path,
                centers_file=centers_file,
                tilt_file=tilt_file,
                spacing_file=spacing_file,
                out_path=out_path,
                show_center=show_center,
                show_tilt=show_tilt,
                show_space=show_space,
            )
            print(f"[INFO] saved: {out_path}")
        except Exception as e:
            print(f"[WARN] visualize failed '{name}': {e}")


In [165]:
visualize_folder(
    image_folder=str(project_root/"craft"/"images"),
    centers_folder=str(project_root/"craft"/"results"/"center"),
    tilts_folder=str(project_root/"craft"/"results"/"tilt_module"),
    spacing_folder=str(project_root/"craft"/"results"/"spacing_module"),
    out_folder=str(project_root/"craft"/"results"/"visualize_1"),
    show_center=True, show_tilt=True, show_space=False,
)

[INFO] saved: D:\Study\학교강의\4학년2학기\캡스톤\Baram_Handwritting_Analysis\craft\results\visualize_1\vis_test1.jpg
[INFO] saved: D:\Study\학교강의\4학년2학기\캡스톤\Baram_Handwritting_Analysis\craft\results\visualize_1\vis_test2.jpg
[INFO] saved: D:\Study\학교강의\4학년2학기\캡스톤\Baram_Handwritting_Analysis\craft\results\visualize_1\vis_test3.jpg
[INFO] saved: D:\Study\학교강의\4학년2학기\캡스톤\Baram_Handwritting_Analysis\craft\results\visualize_1\vis_test4.jpg
[INFO] saved: D:\Study\학교강의\4학년2학기\캡스톤\Baram_Handwritting_Analysis\craft\results\visualize_1\vis_test5.jpg


In [139]:
def score_to_heatmap(score: np.ndarray, colormap=cv2.COLORMAP_JET):
    s = score.astype(np.float32)
    mn, mx = float(np.min(s)), float(np.max(s))
    if mx - mn < 1e-8:
        norm = np.zeros_like(s, dtype=np.uint8)
    else:
        norm = ((s - mn) / (mx - mn) * 255.0).astype(np.uint8)

    heat = cv2.applyColorMap(norm, colormap)  # BGR
    return heat

In [140]:
def overlay_heatmap_on_image(img_bgr: np.ndarray, heat_bgr: np.ndarray, alpha=0.45):
    """
    alpha: heatmap 가중치 (0~1)
    """
    if img_bgr.shape[:2] != heat_bgr.shape[:2]:
        heat_bgr = cv2.resize(heat_bgr, (img_bgr.shape[1], img_bgr.shape[0]), interpolation=cv2.INTER_LINEAR)

    out = cv2.addWeighted(img_bgr, 1.0 - alpha, heat_bgr, alpha, 0)
    return out


In [141]:
def visualize_score_text_folder(
    score_text_folder: str,
    out_folder: str,
    *,
    image_folder: str | None = None, 
    overlay: bool = True,
    alpha: float = 0.45,
    only_heatmap: bool = False,      
):
    os.makedirs(out_folder, exist_ok=True)

    npy_files = [f for f in os.listdir(score_text_folder) if f.startswith("res_") and f.endswith("_score_text.npy")]
    if not npy_files:
        print(f"[WARN] No score_text npy found in: {score_text_folder}")
        return

    for fname in sorted(npy_files):
        name = fname[len("res_") : -len("_score_text.npy")]
        npy_path = os.path.join(score_text_folder, fname)

        score = np.load(npy_path)
        heat = score_to_heatmap(score)

        heat_path = os.path.join(out_folder, f"vis_{name}_heat.jpg")
        imwrite_unicode(heat_path, heat)

        if overlay and (not only_heatmap):
            if image_folder is None:
                print(f"[WARN] image_folder is None → overlay skip for '{name}'")
                continue

            img_path = None
            for ext in [".jpg", ".jpeg", ".png", ".bmp", ".webp"]:
                cand = os.path.join(image_folder, name + ext)
                if os.path.exists(cand):
                    img_path = cand
                    break

            if img_path is None:
                print(f"[WARN] image not found for '{name}' in {image_folder} → overlay skip")
                continue

            img = imread_unicode(img_path)
            if img is None:
                print(f"[WARN] cannot read image '{img_path}' → overlay skip")
                continue

            over = overlay_heatmap_on_image(img, heat, alpha=alpha)
            over_path = os.path.join(out_folder, f"vis_{name}_overlay.jpg")
            imwrite_unicode(over_path, over)

        print(f"[INFO] saved heatmap: {heat_path}")


In [142]:
visualize_score_text_folder(
    score_text_folder=str(project_root/"craft"/"results"/"score_text"),
    out_folder=str(project_root/"craft"/"results"/"visualize_1"),
    image_folder=str(project_root/"craft"/"images"),
    overlay=True,
    alpha=0.45,
)

[INFO] saved heatmap: D:\Study\학교강의\4학년2학기\캡스톤\Baram_Handwritting_Analysis\craft\results\visualize_1\vis_test1_heat.jpg
[INFO] saved heatmap: D:\Study\학교강의\4학년2학기\캡스톤\Baram_Handwritting_Analysis\craft\results\visualize_1\vis_test2_heat.jpg
[INFO] saved heatmap: D:\Study\학교강의\4학년2학기\캡스톤\Baram_Handwritting_Analysis\craft\results\visualize_1\vis_test3_heat.jpg
[INFO] saved heatmap: D:\Study\학교강의\4학년2학기\캡스톤\Baram_Handwritting_Analysis\craft\results\visualize_1\vis_test4_heat.jpg
[INFO] saved heatmap: D:\Study\학교강의\4학년2학기\캡스톤\Baram_Handwritting_Analysis\craft\results\visualize_1\vis_test5_heat.jpg
